In [23]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
import math

pd.set_option('display.max_rows', 100000)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)

folder = r'/Users/gavin/Documents/Metis/Coursework/Project_2/'

# Below is all work related to cleaning addresses to match across dataframes. This Jupyter Notebook only pertains to the info from Douglas Elliman web scrape. Info taken from NYC Open Data cleaned in a different notebook. This is the only way to get location data connected to scraped data. Cleaning requires changing all abreviations to long name format, making everything capital, and removing suffix from numbers. for example:

# 20 W 34th St. becomes 20 W 34 STREET

In [3]:
df=pd.read_csv(folder+'DE_Data_Clean.csv')

In [4]:
df.head()

,Unnamed: 0,URL,Property_Type,MLS/Listing_ID,Full_Street_Address,Neighborhood,ZipCode,Price,Description,Dishwasher,...,Wood,Wooded,on_market_date,Exercise_Room,Half_Bath_(F),Vinyl_(F),Interior_Sq_Ft,Exterior_Acres,Address_Number,Street
0,0,https://www.elliman.com/newyorkcity/sales/deta...,Single Family Home,4419664,720 Point Crescent,Whitestone,11357.0,10000000.0,Click for NYS Housing Discrimination Disclosur...,1.0,...,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,720,Point Crescent
1,1,https://www.elliman.com/newyorkcity/sales/deta...,Single Family Home,3254581,32 Boulevard,Malba,11357.0,8988000.0,Click for NYS Housing Discrimination Disclosur...,NaN,...,NaN,NaN,NaN,NaN,1.0,0.0,7218.0,0.17,32,Boulevard
2,2,https://www.elliman.com/newyorkcity/sales/deta...,Single Family Home,3040190,6-02 158th Street,Beechhurst,11357.0,5999999.0,Click for NYS Housing Discrimination Disclosur...,NaN,...,NaN,NaN,NaN,NaN,2.0,0.0,7500.0,0.50,6-02,158th Street
3,3,https://www.elliman.com/newyorkcity/sales/deta...,Multi Family Townhouse,3214989,42-65 Main Street,Flushing,11355.0,3588000.0,Click for NYS Housing Discrimination Disclosur...,NaN,...,NaN,NaN,NaN,NaN,3.0,0.0,NaN,0.06,42-65,Main Street
4,4,https://www.elliman.com/newyorkcity/sales/deta...,Multi Family Townhouse,4429870,10-22 47th Avenue,Hunters Point,NaN,3500000.0,Click for NYS Housing Discrimination Disclosur...,1.0,...,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,10-22,47th Avenue


In [6]:
df['Street_adj']=df['Street'].str.upper()

In [7]:
df.head()

,Unnamed: 0,URL,Property_Type,MLS/Listing_ID,Full_Street_Address,Neighborhood,ZipCode,Price,Description,Dishwasher,...,Wooded,on_market_date,Exercise_Room,Half_Bath_(F),Vinyl_(F),Interior_Sq_Ft,Exterior_Acres,Address_Number,Street,Street_adj
0,0,https://www.elliman.com/newyorkcity/sales/deta...,Single Family Home,4419664,720 Point Crescent,Whitestone,11357.0,10000000.0,Click for NYS Housing Discrimination Disclosur...,1.0,...,NaN,NaN,NaN,0.0,0.0,NaN,NaN,720,Point Crescent,POINT CRESCENT
1,1,https://www.elliman.com/newyorkcity/sales/deta...,Single Family Home,3254581,32 Boulevard,Malba,11357.0,8988000.0,Click for NYS Housing Discrimination Disclosur...,NaN,...,NaN,NaN,NaN,1.0,0.0,7218.0,0.17,32,Boulevard,BOULEVARD
2,2,https://www.elliman.com/newyorkcity/sales/deta...,Single Family Home,3040190,6-02 158th Street,Beechhurst,11357.0,5999999.0,Click for NYS Housing Discrimination Disclosur...,NaN,...,NaN,NaN,NaN,2.0,0.0,7500.0,0.50,6-02,158th Street,158TH STREET
3,3,https://www.elliman.com/newyorkcity/sales/deta...,Multi Family Townhouse,3214989,42-65 Main Street,Flushing,11355.0,3588000.0,Click for NYS Housing Discrimination Disclosur...,NaN,...,NaN,NaN,NaN,3.0,0.0,NaN,0.06,42-65,Main Street,MAIN STREET
4,4,https://www.elliman.com/newyorkcity/sales/deta...,Multi Family Townhouse,4429870,10-22 47th Avenue,Hunters Point,NaN,3500000.0,Click for NYS Housing Discrimination Disclosur...,1.0,...,NaN,NaN,NaN,0.0,0.0,NaN,NaN,10-22,47th Avenue,47TH AVENUE


In [8]:
df.drop('Unnamed: 0', axis=1, inplace=True)

In [9]:
df['Street_adj']=df['Street_adj'].str.split(' ')

In [10]:
df.head()

,URL,Property_Type,MLS/Listing_ID,Full_Street_Address,Neighborhood,ZipCode,Price,Description,Dishwasher,beds,...,Wooded,on_market_date,Exercise_Room,Half_Bath_(F),Vinyl_(F),Interior_Sq_Ft,Exterior_Acres,Address_Number,Street,Street_adj
0,https://www.elliman.com/newyorkcity/sales/deta...,Single Family Home,4419664,720 Point Crescent,Whitestone,11357.0,10000000.0,Click for NYS Housing Discrimination Disclosur...,1.0,NaN,...,NaN,NaN,NaN,0.0,0.0,NaN,NaN,720,Point Crescent,"[POINT, CRESCENT]"
1,https://www.elliman.com/newyorkcity/sales/deta...,Single Family Home,3254581,32 Boulevard,Malba,11357.0,8988000.0,Click for NYS Housing Discrimination Disclosur...,NaN,6.0,...,NaN,NaN,NaN,1.0,0.0,7218.0,0.17,32,Boulevard,[BOULEVARD]
2,https://www.elliman.com/newyorkcity/sales/deta...,Single Family Home,3040190,6-02 158th Street,Beechhurst,11357.0,5999999.0,Click for NYS Housing Discrimination Disclosur...,NaN,7.0,...,NaN,NaN,NaN,2.0,0.0,7500.0,0.50,6-02,158th Street,"[158TH, STREET]"
3,https://www.elliman.com/newyorkcity/sales/deta...,Multi Family Townhouse,3214989,42-65 Main Street,Flushing,11355.0,3588000.0,Click for NYS Housing Discrimination Disclosur...,NaN,6.0,...,NaN,NaN,NaN,3.0,0.0,NaN,0.06,42-65,Main Street,"[MAIN, STREET]"
4,https://www.elliman.com/newyorkcity/sales/deta...,Multi Family Townhouse,4429870,10-22 47th Avenue,Hunters Point,NaN,3500000.0,Click for NYS Housing Discrimination Disclosur...,1.0,3.0,...,NaN,NaN,NaN,0.0,0.0,NaN,NaN,10-22,47th Avenue,"[47TH, AVENUE]"


In [11]:
df['last']=df['Street_adj'].str[-1]

In [26]:
df['second_last']=df['Street_adj'].str[-2]

In [27]:
df['second']=df['Street_adj'].str[1]

List of suffixes and street names, which prior to cleaning contained all errors/needed changes. Went through by hand to change as error formatting inconsistent.

In [354]:
df['last'].unique()

array(['CRESCENT', 'BOULEVARD', 'STREET', 'AVENUE', 'ROAD', 'LANE',
       'DRIVE', 'PLACE', '217', 'PARKWAY', nan, 'DR', 'AVAILABLE', '251',
       'COURT', 'TERRACE', 'TURNPIKE', 'COURSE', '41', '45', 'EXPRESSWAY',
       '98', 'MARATHON', 'HARDING', '110ST', '71', '46', '123', 'RIDGE',
       '121', 'HARROW', '93ST', '43', '111', 'GILLMORE', '38', '134', '',
       '245', '188', 'WAY', 'CENTRAL', '102', 'SOUTH', '124', '130', '80',
       '69', 'CLINTONVILLE', '199', '79', 'FREEWAY', '157',
       'LAKEVIEW BOULEVARD EAST', '31 STREET', '200 STREET',
       'QUEENS MIDTOWN EXPRESSWAY', 'JORDAN', '147 STREET', '204',
       'MARSDEN', '120 STREET', 'EGMONT PLACE', '224', '171 STREET',
       '96 STREET', 'DUNKIRK STREET', '155 STREET', '198 STREET',
       '3 AVENUE', '160 STREET', 'A804', 'B', 'BROADWAY'], dtype=object)

In [32]:
print(df.iloc[74,0])

https://www.elliman.com/newyorkcity/sales/detail/612-l-615-16_3222076/38-18-217th-bayside-queens-ny-11361


In [362]:
df.loc[df['last']=='B', 'last']= ''

In [238]:
df.loc[df['last']=='', 'address_addendum']= 'STREET'

In [361]:
df.loc[df['last']=='B', 'apt_num']='B'

In [360]:
df[df['last']=='B']

,URL,Property_Type,MLS/Listing_ID,Full_Street_Address,Neighborhood,ZipCode,Price,Description,Dishwasher,beds,Baths,Interior,Year_Built,Patio,Pool,Air_Conditioning,Garage_-_Attached,Water_View,New_Construction,Stucco,Brick,Pre-War,Courtyard,Wood_Heating,Frame,Gardens,Pets_Allowed,Stone,Garage,Concrete_Block,Central_Air_Conditioning,Private_Storage,Terrace,Aluminum_Siding,Cedar_Siding,Storage_Room,Washer_Dryer_Included,Concrete_Siding,Wood,Wooded,on_market_date,Exercise_Room,Half_Bath_(F),Vinyl_(F),Interior_Sq_Ft,Exterior_Acres,Address_Number,Street,Street_adj,last,second_last,second,address_addendum,apt_num
5255,https://www.elliman.com/newyorkcity/sales/deta...,Single Family Home,438429,"13707 68th Drive, B",Northeastern Queens,11367.0,330000.0,Click for NYS Housing Discrimination Disclosur...,NaN,2.0,1.0,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,13707,"68th Drive, B","[68TH, DRIVE,, B]",B,"DRIVE,","DRIVE,",,B


In [221]:
df['apt_num']=""

In [40]:
df.head()

,URL,Property_Type,MLS/Listing_ID,Full_Street_Address,Neighborhood,ZipCode,Price,Description,Dishwasher,beds,Baths,Interior,Year_Built,Patio,Pool,Air_Conditioning,Garage_-_Attached,Water_View,New_Construction,Stucco,Brick,Pre-War,Courtyard,Wood_Heating,Frame,Gardens,Pets_Allowed,Stone,Garage,Concrete_Block,Central_Air_Conditioning,Private_Storage,Terrace,Aluminum_Siding,Cedar_Siding,Storage_Room,Washer_Dryer_Included,Concrete_Siding,Wood,Wooded,on_market_date,Exercise_Room,Half_Bath_(F),Vinyl_(F),Interior_Sq_Ft,Exterior_Acres,Address_Number,Street,Street_adj,last,second_last,second,address_addendum
0,https://www.elliman.com/newyorkcity/sales/deta...,Single Family Home,4419664,720 Point Crescent,Whitestone,11357.0,10000000.0,Click for NYS Housing Discrimination Disclosur...,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,720,Point Crescent,"[POINT, CRESCENT]",CRESCENT,POINT,CRESCENT,
1,https://www.elliman.com/newyorkcity/sales/deta...,Single Family Home,3254581,32 Boulevard,Malba,11357.0,8988000.0,Click for NYS Housing Discrimination Disclosur...,NaN,6.0,5.0,"7,218 \n\t\t\t\tSq Ft.",2009.0,1.0,1.0,1.0,1.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,0.0,7218.0,0.17,32,Boulevard,[BOULEVARD],BOULEVARD,NaN,NaN,
2,https://www.elliman.com/newyorkcity/sales/deta...,Single Family Home,3040190,6-02 158th Street,Beechhurst,11357.0,5999999.0,Click for NYS Housing Discrimination Disclosur...,NaN,7.0,6.0,"7,500 \n\t\t\t\tSq Ft.",2009.0,1.0,1.0,1.0,NaN,1.0,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,0.0,7500.0,0.50,6-02,158th Street,"[158TH, STREET]",STREET,158TH,STREET,
3,https://www.elliman.com/newyorkcity/sales/deta...,Multi Family Townhouse,3214989,42-65 Main Street,Flushing,11355.0,3588000.0,Click for NYS Housing Discrimination Disclosur...,NaN,6.0,2.0,NaN,1968.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,0.0,NaN,0.06,42-65,Main Street,"[MAIN, STREET]",STREET,MAIN,STREET,
4,https://www.elliman.com/newyorkcity/sales/deta...,Multi Family Townhouse,4429870,10-22 47th Avenue,Hunters Point,NaN,3500000.0,Click for NYS Housing Discrimination Disclosur...,1.0,3.0,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,10-22,47th Avenue,"[47TH, AVENUE]",AVENUE,47TH,AVENUE,


In [263]:
print(df.iloc[5109,0])

https://www.elliman.com/newyorkcity/sales/detail/612-l-789-16_3251319/133-07-160th-rochdale-queens-ny-11434


In [203]:
##Individual adds
#last
df.iloc[3,-4]='31 STREET'


In [181]:
#addendum
df.iloc[2805,-1]='STREET'

all 'TH's taken off of end of street numbers:

In [370]:
second_num = df[df['second_last'].str[-2:]=='TH']

In [384]:
second_num['second_last'].value_counts().sum()

2172

In [395]:
test=second_num['second_last'].str[0].isnumeric()

AttributeError: 'Series' object has no attribute 'isnumeric'

In [379]:
final=test.str.isnumeric()

In [393]:
TEST=test[final]

In [404]:
unique_array=second_num['second_last'].unique()

In [409]:
lofchanges=[]
for x in unique_array:
    if x[0].isnumeric():
        lofchanges.append(x)

In [410]:
lofchanges

['158TH',
 '47TH',
 '44TH',
 '45TH',
 '12TH',
 '10TH',
 '244TH',
 '150TH',
 '14TH',
 '86TH',
 '79TH',
 '25TH',
 '20TH',
 '218TH',
 '168TH',
 '205TH',
 '139TH',
 '216TH',
 '97TH',
 '166TH',
 '220TH',
 '56TH',
 '196TH',
 '78TH',
 '118TH',
 '199TH',
 '74TH',
 '214TH',
 '98TH',
 '89TH',
 '107TH',
 '114TH',
 '115TH',
 '254TH',
 '229TH',
 '146TH',
 '159TH',
 '67TH',
 '95TH',
 '39TH',
 '64TH',
 '77TH',
 '88TH',
 '60TH',
 '266TH',
 '135TH',
 '112TH',
 '215TH',
 '194TH',
 '84TH',
 '87TH',
 '230TH',
 '90TH',
 '256TH',
 '75TH',
 '167TH',
 '227TH',
 '148TH',
 '144TH',
 '145TH',
 '116TH',
 '70TH',
 '69TH',
 '160TH',
 '27TH',
 '120TH',
 '46TH',
 '165TH',
 '109TH',
 '111TH',
 '164TH',
 '68TH',
 '249TH',
 '76TH',
 '40TH',
 '24TH',
 '5TH',
 '35TH',
 '149TH',
 '80TH',
 '126TH',
 '207TH',
 '65TH',
 '49TH',
 '100TH',
 '59TH',
 '106TH',
 '36TH',
 '38TH',
 '34TH',
 '104TH',
 '6TH',
 '29TH',
 '85TH',
 '7TH',
 '210TH',
 '48TH',
 '66TH',
 '138TH',
 '212TH',
 '136TH',
 '156TH',
 '108TH',
 '147TH',
 '37TH',
 '15

In [411]:
for i in lofchanges:
    j=i[:-2]
    df.loc[df['second_last']==i, 'second_last']=j

In [400]:
list=second_num[second_num['second_last'] not in outlist]

ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

In [388]:
df_copy=df.copy()

In [389]:
df.loc[df['second_last'] == final]

ValueError: Can only compare identically-labeled Series objects

In [ ]:
df.loc[df['second_last']=='B', 'second_last']= ''

In [427]:
df[df['second']=='TH']

,URL,Property_Type,MLS/Listing_ID,Full_Street_Address,Neighborhood,ZipCode,Price,Description,Dishwasher,beds,Baths,Interior,Year_Built,Patio,Pool,Air_Conditioning,Garage_-_Attached,Water_View,New_Construction,Stucco,Brick,Pre-War,Courtyard,Wood_Heating,Frame,Gardens,Pets_Allowed,Stone,Garage,Concrete_Block,Central_Air_Conditioning,Private_Storage,Terrace,Aluminum_Siding,Cedar_Siding,Storage_Room,Washer_Dryer_Included,Concrete_Siding,Wood,Wooded,on_market_date,Exercise_Room,Half_Bath_(F),Vinyl_(F),Interior_Sq_Ft,Exterior_Acres,Address_Number,Street,Street_adj,last,second_last,second,address_addendum,apt_num
216,https://www.elliman.com/newyorkcity/sales/deta...,Single Family Home,3215606,172-03 35 Th Avenue,Flushing,11358.0,2999000.0,Click for NYS Housing Discrimination Disclosur...,NaN,6.0,3.0,"6,650 \n\t\t\t\tSq Ft.",2005.0,NaN,NaN,1.0,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,0.0,6650.0,0.13,172-03,35 Th Avenue,"[35, TH, AVENUE]",AVENUE,TH,TH,,
694,https://www.elliman.com/newyorkcity/sales/deta...,Multi Family Townhouse,3191151,34-38 12 Th,Astoria,11106.0,1499000.0,Click for NYS Housing Discrimination Disclosur...,NaN,5.0,4.0,"2,523 \n\t\t\t\tSq Ft.",2005.0,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,2523.0,0.06,34-38,12 Th,"[12, TH]",STREET,12,TH,,
859,https://www.elliman.com/newyorkcity/sales/deta...,Multi Family Townhouse,3204133,164-23 77 Th Avenue,Hillcrest,11366.0,1398888.0,Click for NYS Housing Discrimination Disclosur...,NaN,4.0,5.0,"2,256 \n\t\t\t\tSq Ft.",2006.0,1.0,NaN,1.0,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,2256.0,0.09,164-23,77 Th Avenue,"[77, TH, AVENUE]",AVENUE,TH,TH,,
2280,https://www.elliman.com/newyorkcity/sales/deta...,Single Family Home,3126782,22650 77 th Avenue,Oakland Gardens,11364.0,938000.0,Click for NYS Housing Discrimination Disclosur...,NaN,3.0,2.0,NaN,1950.0,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,0.12,22650,77 th Avenue,"[77, TH, AVENUE]",AVENUE,TH,TH,,
2598,https://www.elliman.com/newyorkcity/sales/deta...,Single Family Home,3233002,144-29 68 Th Drive,Northeastern Queens,11367.0,888000.0,Click for NYS Housing Discrimination Disclosur...,NaN,3.0,2.0,NaN,1950.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,0.0,NaN,0.04,144-29,68 Th Drive,"[68, TH, DRIVE]",DRIVE,TH,TH,,
2829,https://www.elliman.com/newyorkcity/sales/deta...,Single Family Home,3173001,218-11 94 Th Drive,Queens Village,11427.0,849900.0,Click for NYS Housing Discrimination Disclosur...,NaN,4.0,3.0,NaN,2009.0,1.0,NaN,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,0.09,218-11,94 Th Drive,"[94, TH, DRIVE]",DRIVE,TH,TH,,


In [414]:
df.head(1000)

,URL,Property_Type,MLS/Listing_ID,Full_Street_Address,Neighborhood,ZipCode,Price,Description,Dishwasher,beds,Baths,Interior,Year_Built,Patio,Pool,Air_Conditioning,Garage_-_Attached,Water_View,New_Construction,Stucco,Brick,Pre-War,Courtyard,Wood_Heating,Frame,Gardens,Pets_Allowed,Stone,Garage,Concrete_Block,Central_Air_Conditioning,Private_Storage,Terrace,Aluminum_Siding,Cedar_Siding,Storage_Room,Washer_Dryer_Included,Concrete_Siding,Wood,Wooded,on_market_date,Exercise_Room,Half_Bath_(F),Vinyl_(F),Interior_Sq_Ft,Exterior_Acres,Address_Number,Street,Street_adj,last,second_last,second,address_addendum,apt_num
0,https://www.elliman.com/newyorkcity/sales/deta...,Single Family Home,4419664,720 Point Crescent,Whitestone,11357.0,10000000.0,Click for NYS Housing Discrimination Disclosur...,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,720,Point Crescent,"[POINT, CRESCENT]",CRESCENT,POINT,CRESCENT,,
1,https://www.elliman.com/newyorkcity/sales/deta...,Single Family Home,3254581,32 Boulevard,Malba,11357.0,8988000.0,Click for NYS Housing Discrimination Disclosur...,NaN,6.0,5.0,"7,218 \n\t\t\t\tSq Ft.",2009.0,1.0,1.0,1.0,1.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,0.0,7218.0,0.17,32,Boulevard,[BOULEVARD],BOULEVARD,NaN,NaN,,
2,https://www.elliman.com/newyorkcity/sales/deta...,Single Family Home,3040190,6-02 158th Street,Beechhurst,11357.0,5999999.0,Click for NYS Housing Discrimination Disclosur...,NaN,7.0,6.0,"7,500 \n\t\t\t\tSq Ft.",2009.0,1.0,1.0,1.0,NaN,1.0,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,0.0,7500.0,0.50,6-02,158th Street,"[158TH, STREET]",STREET,158,STREET,,
3,https://www.elliman.com/newyorkcity/sales/deta...,Multi Family Townhouse,3214989,42-65 Main Street,Flushing,11355.0,3588000.0,Click for NYS Housing Discrimination Disclosur...,NaN,6.0,2.0,NaN,1968.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,0.0,NaN,0.06,42-65,Main Street,"[MAIN, STREET]",STREET,MAIN,STREET,,
4,https://www.elliman.com/newyorkcity/sales/deta...,Multi Family Townhouse,4429870,10-22 47th Avenue,Hunters Point,NaN,3500000.0,Click for NYS Housing Discrimination Disclosur...,1.0,3.0,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,10-22,47th Avenue,"[47TH, AVENUE]",AVENUE,47,AVENUE,,
5,https://www.elliman.com/newyorkcity/sales/deta...,Multi Family Townhouse,4429868,10-20 47th Avenue,Long Island City,NaN,3500000.0,Click for NYS Housing Discrimination Disclosur...,1.0,5.0,3.0,"2,210 \n\t\t\t\tSq Ft.",1901.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,2210.0,NaN,10-20,47th Avenue,"[47TH, AVENUE]",AVENUE,47,AVENUE,,
6,https://www.elliman.com/newyorkcity/sales/deta...,Investment,4200137,45-43 21st St,Long Island City,NaN,2980000.0,Click for NYS Housing Discrimination Disclosur...,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,45-43,21st St,"[21ST, ST]",STREET,21ST,ST,,
7,https://www.elliman.com/newyorkcity/sales/deta...,Single Family Home,3225573,6 Beverly Road,Douglaston,11363.0,2950000.0,Click for NYS Housing Discrimination Disclosur...,NaN,6.0,3.0,"4,700 \n\t\t\t\tSq Ft.",1909.0,1.0,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,0.0,4700.0,0.23,6,Beverly Road,"[BEVERLY, ROAD]",ROAD,BEVERLY,ROAD,,
8,https://www.elliman.com/newyorkcity/sales/deta...,Multi Family Townhouse,4081249,5-13 51st Avenue,Long Island City,NaN,2749000.0,Click for NYS Housing Discrimination Disclosur...,1.0,6.0,6.0,"3,600 \n\t\t\t\tSq Ft.",NaN,NaN,NaN,

More cleaning, for second word in each address (word immediately proceeding the address):

In [415]:
df['second'].unique()

array(['CRESCENT', nan, 'STREET', 'AVENUE', 'ST', 'ROAD', 'BOULEVARD',
       'LANE', 'POND', 'FRONT', 'DRIVE', 'PLACE', '131ST', 'HAMPTON',
       '118TH', 'PARKWAY', 'AVE', 'CHANNEL', 'PL', 'CENTRAL', 'LEWIS',
       'NECK', 'CONDUIT', 'BAY', 'BLVD', 'ST.', 'TH', '57', 'DR',
       'OLIVET', 'RD', 'AVAILABLE', 'COVE', 'PKWAY', '100TH', 'PKWY',
       '132', 'COURT', 'ZANDT', 'TERRACE', 'TURNPIKE', 'NICHOLAS', 'TER',
       '36', 'CRSE', '151', 'EDGE', 'CORNERS', 'MEMORIAL', 'DOREN',
       'KENNEDY', 'HARDING', '144', 'BEACH', '135', 'BLVD.', '65', 'THE',
       'ISLAND', 'AVE.', '5TH', 'WYCK', 'RD.', 'RIDGE', 'HALL', '65TH',
       'R', '12TH', 'LN', 'HARBOR', '131', '139TH', 'POINT', 'MEADOW',
       'FELIX', '68', 'B', '91ST', '91', '140', 'BREEZE', 'OR', '135TH',
       'EXPRESSWAY', '23RD', 'ALTO', 'DUNE', 'JAMES', 'AVE.SOUTH', '98',
       '119TH', '74TH', 'EXPY', 'REEF', '32ND', 'VALLEY', 'AIRMEN', '',
       'HILLS', 'CREEK', '101ST', '101', '178TH', '21ST', '77', '117TH',
  

In [421]:
filt = df[df['second'].str[-2:]=='TH']
filt.head()

,URL,Property_Type,MLS/Listing_ID,Full_Street_Address,Neighborhood,ZipCode,Price,Description,Dishwasher,beds,Baths,Interior,Year_Built,Patio,Pool,Air_Conditioning,Garage_-_Attached,Water_View,New_Construction,Stucco,Brick,Pre-War,Courtyard,Wood_Heating,Frame,Gardens,Pets_Allowed,Stone,Garage,Concrete_Block,Central_Air_Conditioning,Private_Storage,Terrace,Aluminum_Siding,Cedar_Siding,Storage_Room,Washer_Dryer_Included,Concrete_Siding,Wood,Wooded,on_market_date,Exercise_Room,Half_Bath_(F),Vinyl_(F),Interior_Sq_Ft,Exterior_Acres,Address_Number,Street,Street_adj,last,second_last,second,address_addendum,apt_num
89,https://www.elliman.com/newyorkcity/sales/deta...,Single Family Home,3817806,174 Beach 118th St,Rockaway Beach,NaN,929000.0,Click for NYS Housing Discrimination Disclosur...,1.0,6.0,3.0,"6,000 \n\t\t\t\tSq Ft.",1905.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,6000.0,NaN,174,Beach 118th St,"[BEACH, 118TH, ST]",STREET,118,118TH,,
216,https://www.elliman.com/newyorkcity/sales/deta...,Single Family Home,3215606,172-03 35 Th Avenue,Flushing,11358.0,2999000.0,Click for NYS Housing Discrimination Disclosur...,NaN,6.0,3.0,"6,650 \n\t\t\t\tSq Ft.",2005.0,NaN,NaN,1.0,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,0.0,6650.0,0.13,172-03,35 Th Avenue,"[35, TH, AVENUE]",AVENUE,TH,TH,,
347,https://www.elliman.com/newyorkcity/sales/deta...,Multi Family Townhouse,3225695,224 Beach 100th Street,Seaside,11694.0,1999999.0,Click for NYS Housing Discrimination Disclosur...,NaN,10.0,10.0,NaN,1930.0,1.0,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,0.11,224,Beach 100th Street,"[BEACH, 100TH, STREET]",STREET,100,100TH,,
694,https://www.elliman.com/newyorkcity/sales/deta...,Multi Family Townhouse,3191151,34-38 12 Th,Astoria,11106.0,1499000.0,Click for NYS Housing Discrimination Disclosur...,NaN,5.0,4.0,"2,523 \n\t\t\t\tSq Ft.",2005.0,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,2523.0,0.06,34-38,12 Th,"[12, TH]",STREET,12,TH,,
859,https://www.elliman.com/newyorkcity/sales/deta...,Multi Family Townhouse,3204133,164-23 77 Th Avenue,Hillcrest,11366.0,1398888.0,Click for NYS Housing Discrimination Disclosur...,NaN,4.0,5.0,"2,256 \n\t\t\t\tSq Ft.",2006.0,1.0,NaN,1.0,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,2256.0,0.09,164-23,77 Th Avenue,"[77, TH, AVENUE]",AVENUE,TH,TH,,


In [423]:
unique_array=filt['second'].unique()
unique_array

array(['118TH', 'TH', '100TH', '5TH', '65TH', '12TH', '139TH', '135TH',
       'AVE.SOUTH', '119TH', '74TH', '178TH', '117TH', '47TH', '164TH',
       '44TH', '9TH', '218TH', '88TH', '7TH', '66TH', '115TH', '29TH',
       '27TH', '67TH', '37TH', '64TH', '69TH', '46TH', '8TH', '30TH',
       '11TH', '13TH', '68TH', '228TH', '45TH', '60TH', '26TH'],
      dtype=object)

In [424]:
lofchanges=[]
for x in unique_array:
    if x[0].isnumeric():
        lofchanges.append(x)

In [425]:
lofchanges

['118TH',
 '100TH',
 '5TH',
 '65TH',
 '12TH',
 '139TH',
 '135TH',
 '119TH',
 '74TH',
 '178TH',
 '117TH',
 '47TH',
 '164TH',
 '44TH',
 '9TH',
 '218TH',
 '88TH',
 '7TH',
 '66TH',
 '115TH',
 '29TH',
 '27TH',
 '67TH',
 '37TH',
 '64TH',
 '69TH',
 '46TH',
 '8TH',
 '30TH',
 '11TH',
 '13TH',
 '68TH',
 '228TH',
 '45TH',
 '60TH',
 '26TH']

In [426]:
for i in lofchanges:
    j=i[:-2]
    df.loc[df['second']==i, 'second']=j

In [428]:
df.to_csv(folder+'DE_cleaning_process.csv')